In [12]:
import numpy as np
import datetime
import pandas as pd

import time
import pickle
import os
from pathlib import Path

from math import log10, floor

import matplotlib.pyplot as plt
import matplotlib



In [129]:
class Country(object):
    def __init__(self,name,iso):
        self.name = name
        self.iso = iso
        
        # number of cases per day
        self.raw_data = {"cases":None,"deaths":None,"hospitalized":None,"ICU":None}
        self.data = {}
        
        # Different hooks along the pre-processing
        self.Load()
        self.Process()
        self.GenWeeklySeries()
        # Now contains a weekly dataseries
        
    def Load(self):
        pass
    
    def Process(self):
        pass
    
    def GenWeeklySeries(self,data=None):
        if data == None:
            data = self.data
            for k,v in data.items():
                print(iso,k,v.columns)
    #    print(self.iso,self.raw_data)
        
    def IsAvailable(self,key):
        return self.data.get(key,None) != None
                
class BasicCountry(Country):
    def __init__(self,name,iso):
        super(BasicCountry,self).__init__(name,iso)
        
    def Load(self):
        for k in self.raw_data.keys():
            fn_full = "../DataByAge/"+iso+"/"+k+".csv"
            try:
                df = pd.read_csv(fn_full,header=0,index_col=0)
                self.raw_data[k] = df
            except:
                pass
            
    def Process(self,data=None):
        if data == None:
            data = self.raw_data
            
        replace = {"80-":"80+"}
        unused = ["confirmed","deaths"]
        for k,v in data.items():
            if type(v) != type(None):
                columns = v.columns
            
                def replace_index(r):
                    if r in replace.keys():
                        r = replace[r]
                    return r
            
                columns = map(replace_index,columns)
                
                
                def remove_unused(r):
                    for u in unused:
                        if u in r:
                            r = r.replace(u,"").lstrip()
                    return r
                
                v.columns = map(remove_unused,columns)
                
                self.data[k] = v
                
class RemoveNamedDoubleIndex(BasicCountry):
    def __init__(self,name,iso):
        super(RemoveNamedDoubleIndex,self).__init__(name,iso)
        
    def Process(self):
        for k,v in self.raw_data.items():
            if type(v) != type(None):
                columns = v.columns
                
                def remove_double_index(r):
                    if "(" in r and ")" in r and "," in r:
                        r = r.split("'")[3]
                    return r
                        
                v.columns = map(remove_double_index,columns)
        super(RemoveNamedDoubleIndex,self).Process(self.raw_data)
    
class Swizerland(BasicCountry):
    def __init__(self,name,iso):
        super(Swizerland,self).__init__(name,iso)
        
    def Process(self):
        data = {}
        for k,v in self.raw_data.items():
            if type(v) != type(None):
                d = v.diff(axis=0)
                
                d = d.iloc[1:]
                data[k] = d
        super(Swizerland,self).Process(data)
        
class Germany(Country):
    def __init__(self,name,iso):
        super(Germany,self).__init__(name,iso)
        
    def Load(self):
        pass

In [130]:
cntrs = {}
#cntrs["CZ"] = {"name":"Czech Republic","class":BasicCountry}
cntrs["NL"] = {"name":"Netherlands","class":BasicCountry}
cntrs["PRT"] = {"name":"Portugal","class":RemoveNamedDoubleIndex}
cntrs["BE"] = {"name":"Belgium","class":RemoveNamedDoubleIndex}

cntrs["DE"] = {"name":"Germany","class":Germany}
cntrs["CHE"] = {"name":"Swizerland","class":Swizerland}

countries = {}
for iso,cntry in cntrs.items():
    countries[iso] = cntry["class"](cntry["name"],iso)


print(countries)

NL cases Index(['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
       '80-89', '90+', '<50', 'Unknown'],
      dtype='object')
NL deaths Index(['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
       '80-89', '90+', '<50', 'Unknown'],
      dtype='object')
PRT cases Index(['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
       '80+'],
      dtype='object')
PRT deaths Index(['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
       '80+'],
      dtype='object')
BE cases Index(['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
       '80-89', '90+'],
      dtype='object')
BE deaths Index(['0-24', '25-44', '45-64', '65-74', '75-84', '85+'], dtype='object')
CHE cases Index(['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
       '80+'],
      dtype='object')
CHE deaths Index(['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
       '80+'],
      